## Part II: `Candidate` Extraction

In [1]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ corpus snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading the `Corpus`

First, we will load the `Corpus` that we preprocessed in Part I:

In [3]:
from snorkel.models import Corpus

corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Training').one()
print "%s contains %d Documents" % (corpus, len(corpus))

Corpus (Hardware Training) contains 75 Documents


## Defining a `Candidate` Schema

In [4]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

## Writing a basic `CandidateExtractor`

In [5]:
from snorkel.candidates import OmniNgrams

omni_ngrams = OmniNgrams(n_max=3, split_tokens=None)

In [6]:
from hardware_utils import load_extended_parts_dict

filename='data/hardware/hardware_gold.csv'
parts_dict = load_extended_parts_dict(filename)
print "Loaded %d part numbers." % len(parts_dict)

Loaded 5331 part numbers.


In [7]:
from snorkel.matchers import DictionaryMatch, RegexMatchSpan

parts_matcher = DictionaryMatch(d=parts_dict, ignore_case=True)
temp_matcher = RegexMatchSpan(rgx=ur'-\s?[5-7][05]')

In [8]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part_Temp, [omni_ngrams, omni_ngrams], [parts_matcher, temp_matcher])

## Running the `CandidateExtractor`

In [9]:
%time candidates = ce.extract(corpus.documents, 'Hardware Training Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

[========================================] 100%

CPU times: user 22.4 s, sys: 883 ms, total: 23.3 s
Wall time: 23.1 s
Candidate Set (Hardware Training Candidates) contains 319 Candidates


In [10]:
for c in candidates[:5]:
    print c

Part_Temp(Span("BC337-16", parent=3898, chars=[0,7], words=[0,0]), Span("-55", parent=3927, chars=[22,24], words=[3,3]))
Part_Temp(Span("BC337-16", parent=3898, chars=[0,7], words=[0,0]), Span("-55", parent=3922, chars=[24,26], words=[3,3]))
Part_Temp(Span("BC337", parent=3981, chars=[21,25], words=[3,3]), Span("-55", parent=3922, chars=[24,26], words=[3,3]))
Part_Temp(Span("BC338", parent=3999, chars=[20,24], words=[3,3]), Span("-55", parent=3922, chars=[24,26], words=[3,3]))
Part_Temp(Span("BC338-16", parent=3898, chars=[15,22], words=[3,3]), Span("-55", parent=3927, chars=[22,24], words=[3,3]))


### Saving the extracted candidates

In [11]:
session.add(candidates)
session.commit()

### Reloading the candidates

In [12]:
from snorkel.models import CandidateSet

candidates = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()
print "%s contains %d Candidates" % (candidates, len(candidates))

Candidate Set (Hardware Training Candidates) contains 319 Candidates


### Repeating for development and test corpora

In [13]:
for corpus_name in ['Hardware Development', 'Hardware Test']:
    corpus = session.query(Corpus).filter(Corpus.name == corpus_name).one()
    %time candidates = ce.extract(corpus.documents, corpus_name + ' Candidates', session)
    session.add(c)
    print "%s contains %d Candidates" % (candidates, len(candidates))
session.commit()

[========================================] 100%

CPU times: user 5.89 s, sys: 177 ms, total: 6.06 s
Wall time: 6.01 s
Candidate Set (Hardware Development Candidates) contains 313 Candidates


CPU times: user 5.95 ms, sys: 1.04 ms, total: 6.99 ms
Wall time: 7.72 ms
Candidate Set (Hardware Test Candidates) contains 0 Candidates


In [14]:
# If necessary
# import os
# os.system('cp snorkel.db snorkel.db\ candidates');

Next, in Part 3, we will load `Labels` for each of our `Candidates` so that we can evaluate performance.